# Altair in `JupyterLite`

**Altair** is a declarative statistical visualization library for Python.

Most of the examples below are from: https://altair-viz.github.io/gallery

## Import the dependencies:

In [ ]:
%pip install -q altair

## Simple Bar Chart

In [ ]:
import altair as alt
import pandas as pd

source = pd.DataFrame(
    {
        "a": ["A", "B", "C", "D", "E", "F", "G", "H", "I"],
        "b": [28, 55, 43, 91, 81, 53, 19, 87, 52],
    }
)

alt.Chart(source).mark_bar().encode(x="a", y="b")

## Simple Heatmap

In [ ]:
import altair as alt
import numpy as np
import pandas as pd

# Compute x^2 + y^2 across a 2D grid
x, y = np.meshgrid(range(-5, 5), range(-5, 5))
z = x**2 + y**2

# Convert this grid to columnar data expected by Altair
source = pd.DataFrame({"x": x.ravel(), "y": y.ravel(), "z": z.ravel()})

alt.Chart(source).mark_rect().encode(x="x:O", y="y:O", color="z:Q")

## Install the Vega Dataset

In [ ]:
%pip install -q vega-datasets

## Interactive Average

In [ ]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()
brush = alt.selection(type="interval", encodings=["x"])

bars = (
    alt.Chart()
    .mark_bar()
    .encode(
        x="month(date):O",
        y="mean(precipitation):Q",
        opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
    )
    .add_selection(brush)
)

line = (
    alt.Chart()
    .mark_rule(color="firebrick")
    .encode(y="mean(precipitation):Q", size=alt.SizeValue(3))
    .transform_filter(brush)
)

alt.layer(bars, line, data=source)

## Locations of US Airports

In [ ]:
import altair as alt
from vega_datasets import data

airports = data.airports.url
states = alt.topo_feature(data.us_10m.url, feature="states")

# US states background
background = (
    alt.Chart(states)
    .mark_geoshape(fill="lightgray", stroke="white")
    .properties(width=500, height=300)
    .project("albersUsa")
)

# airport positions on background
points = (
    alt.Chart(airports)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Number of Airports"),
        color=alt.value("steelblue"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title="Number of airports in US")
)

background + points